In [ ]:
from faster_whisper import WhisperModel

#Takes a .wav file for processing
model = WhisperModel("small.en", device="cpu", compute_type="int8")

segments, info = model.transcribe(
    "kokoro_out.wav",
    beam_size=5,          # good quality/speed tradeoff
    vad_filter=True       # skip long silences
)

print(f"Detected language: {info.language}  (p={info.language_probability:.2f})")
for s in segments:
    print(f"[{s.start:.2f} → {s.end:.2f}] {s.text}")

text = " ".join(s.text for s in segments)


print("\nFull text:\n", text)


In [1]:
#!/usr/bin/env python3
import os, json, sys, datetime as dt
from pathlib import Path

from dotenv import load_dotenv
import dateparser
import google.generativeai as genai

from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build

# ---------- Config ----------
SCOPES = [
    "https://www.googleapis.com/auth/calendar.readonly",
    "https://www.googleapis.com/auth/calendar.events",
]
TOKEN_PATH = Path("token.json")  # per-user token for this CLI
DEFAULT_TZ = "Pacific/Auckland"  # change if you like

# ---------- OAuth helpers ----------
def _client_config_from_env():
    load_dotenv()
    raw = os.getenv("CALENDAR_KEY")
    if not raw:
        raise RuntimeError("Missing CALENDAR_KEY in .env")
    try:
        return json.loads(raw)   # {"installed": {...}} or {"web": {...}}
    except json.JSONDecodeError as e:
        raise RuntimeError("CALENDAR_KEY is not valid JSON") from e

def ensure_creds():
    """Create/refresh token.json via OAuth (Desktop flow) and return Credentials."""
    creds = None
    if TOKEN_PATH.exists():
        creds = Credentials.from_authorized_user_file(str(TOKEN_PATH), SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_config(_client_config_from_env(), SCOPES)
            # This opens a browser to approve once; then token.json is saved.
            creds = flow.run_local_server(port=0)
        TOKEN_PATH.write_text(creds.to_json())
    return creds

def svc_gcal():
    creds = ensure_creds()
    return build("calendar", "v3", credentials=creds, cache_discovery=False)

def user_timezone(service):
    try:
        tz = service.settings().get(setting="timezone").execute().get("value")
        return tz or DEFAULT_TZ
    except Exception:
        return DEFAULT_TZ

# ---------- Calendar tools ----------
def gcal_list_events(start_iso: str, end_iso: str, calendar_id: str = "primary"):
    svc = svc_gcal()
    resp = svc.events().list(
        calendarId=calendar_id,
        timeMin=start_iso, timeMax=end_iso,
        singleEvents=True, orderBy="startTime", maxResults=50
    ).execute()
    items = resp.get("items", [])
    out = []
    for e in items:
        out.append({
            "id": e["id"],
            "summary": e.get("summary", "(no title)"),
            "start": e["start"].get("dateTime") or e["start"].get("date"),
            "end":   e["end"].get("dateTime") or e["end"].get("date"),
            "htmlLink": e.get("htmlLink"),
        })
    return out

def gcal_create_event(title: str, start_iso: str, end_iso: str,
                      tz: str | None = None, calendar_id: str = "primary"):
    svc = svc_gcal()
    tz = tz or user_timezone(svc)
    event = {
        "summary": title,
        "start": {"dateTime": start_iso, "timeZone": tz},
        "end":   {"dateTime": end_iso,   "timeZone": tz},
    }
    created = svc.events().insert(calendarId=calendar_id, body=event).execute()
    return {"id": created["id"], "htmlLink": created.get("htmlLink"), "summary": created.get("summary")}

# ---------- Gemini tool wiring ----------
def gemini_model():
    load_dotenv()
    genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

    tools = [{
      "function_declarations": [
        {
          "name": "gcal_list_events",
          "description": "List calendar events in a time window (ISO datetimes).",
          "parameters": {
            "type": "OBJECT",
            "properties": {
              "start_iso": {"type":"STRING"},
              "end_iso":   {"type":"STRING"},
              "calendar_id":{"type":"STRING","nullable":True}
            },
            "required": ["start_iso","end_iso"]
          }
        },
        {
          "name": "gcal_create_event",
          "description": "Create a calendar event.",
          "parameters": {
            "type": "OBJECT",
            "properties": {
              "title": {"type":"STRING"},
              "start_iso":{"type":"STRING"},
              "end_iso":{"type":"STRING"},
              "tz":{"type":"STRING","nullable":True},
              "calendar_id":{"type":"STRING","nullable":True}
            },
            "required": ["title","start_iso","end_iso"]
          }
        }
      ]
    }]

    return genai.GenerativeModel(
        model_name="gemini-2.5-flash-lite",
        tools=tools,
        system_instruction=(
            "You are a concise scheduling assistant for a single user. "
            "When the user asks to list/create events, call the right function. "
            "Prefer ISO-8601 datetimes (e.g., 2025-10-07T15:00:00+13:00). "
            "If the user is ambiguous, ask a brief clarifying question."
        )
    )

def maybe_parse_iso(text: str) -> str | None:
    """Fallback: if Gemini returns a non-ISO date in tool args, try parsing it."""
    if not text: return None
    if "T" in text and ":" in text:
        return text  # likely already ISO
    dt_parsed = dateparser.parse(text)
    if not dt_parsed: return None
    # assume local timezone; convert to ISO with offset
    return dt_parsed.isoformat()

def run_once(user_text: str) -> str:
    model = gemini_model()
    first = model.generate_content(user_text)

    # Did the model ask to call a function?
    fcall = None
    for part in first.candidates[0].content.parts:
        if getattr(part, "function_call", None):
            fcall = part.function_call
            break

    if not fcall:
        return first.text

    name = fcall.name
    args = {k: v for k, v in fcall.args.items()}

    # sanitize/parse any times if needed
    if "start_iso" in args:
        args["start_iso"] = maybe_parse_iso(args["start_iso"]) or args["start_iso"]
    if "end_iso" in args:
        args["end_iso"]   = maybe_parse_iso(args["end_iso"])   or args["end_iso"]

    if name == "gcal_list_events":
        result = gcal_list_events(**args)
    elif name == "gcal_create_event":
        result = gcal_create_event(**args)
    else:
        result = {"error": f"Unknown tool {name}"}

    tool_msg = {"function_response": {"name": name, "response": result}}
    final = model.generate_content([user_text, tool_msg])
    return final.text

# ---------- REPL ----------
def main():
    print("Gemini + Google Calendar CLI. Type 'exit' to quit.")
    while True:
        try:
            user = input("You> ").strip()
        except (EOFError, KeyboardInterrupt):
            print()
            break
        if not user: 
            continue
        if user.lower() in {"exit", "quit"}:
            break
        try:
            reply = run_once(user)
            print("Bot>", reply)
        except Exception as e:
            print("ERR>", e, file=sys.stderr)

if __name__ == "__main__":
    main()


Gemini + Google Calendar CLI. Type 'exit' to quit.


/Users/marcus/Documents/GitHub/CTRL-Time/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
E0000 00:00:1759651336.974279 14875709 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


Bot> I can help you schedule that event. Could you please specify the exact date and time for the event, including the year and time zone?


E0000 00:00:1759651363.080005 14875709 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


Bot> What is the event title and how long will it be?


...

Bot> For what date and time zone should I set this reminder?


E0000 00:00:1759651399.455253 14875709 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


Bot> I'm not sure what you mean by "NZ". Could you please clarify?


E0000 00:00:1759651403.322599 14875709 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


Bot> Are you trying to set your location to New Zealand? Or do you want to know what the weather is like in New Zealand? Or something else?



In [ ]:
# --- put near your other imports ---
import os, json, sys
import google.generativeai as genai
from dotenv import load_dotenv

# your existing tool fns:
# from tools_gcal import gcal_list_events, gcal_create_event

# --- model + tools setup (once) ---
load_dotenv()
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

tools = [{
  "function_declarations": [
    {
      "name": "gcal_list_events",
      "description": "List calendar events in a time window (ISO datetimes).",
      "parameters": {
        "type": "OBJECT",
        "properties": {
          "start_iso": {"type":"STRING"},
          "end_iso":   {"type":"STRING"},
          "calendar_id":{"type":"STRING","nullable":True}
        },
        "required": ["start_iso","end_iso"]
      }
    },
    {
      "name": "gcal_create_event",
      "description": "Create a calendar event.",
      "parameters": {
        "type": "OBJECT",
        "properties": {
          "title": {"type":"STRING"},
          "start_iso":{"type":"STRING"},
          "end_iso":{"type":"STRING"},
          "tz":{"type":"STRING","nullable":True},
          "calendar_id":{"type":"STRING","nullable":True}
        },
        "required": ["title","start_iso","end_iso"]
      }
    }
  ]
}]

model = genai.GenerativeModel(
  model_name="models/gemini-2.5-flash",   # use a model your key can access
  tools=tools,
  system_instruction=(
    "You are a scheduling assistant. "
    "Prefer calling the provided functions instead of asking multiple questions. "
    "If you need missing info, ask exactly one short question. "
    "Use ISO-8601 for datetimes. If the user says 'NZ', assume Pacific/Auckland."
  )
)

# --- dispatcher to call your Python tools ---
def call_tool(name: str, args: dict):
    if name == "gcal_list_events":
        return gcal_list_events(**args)
    if name == "gcal_create_event":
        # simple NZ alias
        if args.get("tz", "").lower() in {"nz","nzdt","nzt","new zealand"}:
            args["tz"] = "Pacific/Auckland"
        return gcal_create_event(**args)
    return {"error": f"unknown tool {name}"}

# --- STATEFUL REPL WITH TOOL LOOP ---
def run_chat():
    chat = model.start_chat(history=[])   # <-- keeps context across turns
    print("Gemini + Google Calendar CLI. Type 'exit' to quit.")

    while True:
        try:
            user = input("You> ").strip()
        except (EOFError, KeyboardInterrupt):
            print()
            break
        if not user:
            continue
        if user.lower() in {"exit", "quit"}:
            break

        # 1) send to Gemini
        resp = chat.send_message(user)

        # 2) handle tool-calls until Gemini replies with text
        while True:
            parts = resp.candidates[0].content.parts
            fcall = next((p.function_call for p in parts if getattr(p, "function_call", None)), None)

            if not fcall:
                # plain text answer
                print("Bot>", resp.text)
                break

            # execute the requested tool server-side
            result = call_tool(fcall.name, dict(fcall.args))

            # feed the function result back into the SAME chat
            resp = chat.send_message({
                "function_response": {
                    "name": fcall.name,
                    "response": result
                }
            })

if __name__ == "__main__":
    run_chat()


Gemini + Google Calendar CLI. Type 'exit' to quit.


E0000 00:00:1759651624.165635 14875709 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


Bot> I can do that. What is the current date?
Bot> OK. I've scheduled your event for tomorrow at 6pm.


In [4]:
import os, google.generativeai as genai
from dotenv import load_dotenv

load_dotenv()
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

print("google-generativeai version:", genai.__version__)

# Show every model your key can see
for m in genai.list_models():
    print(m.name, "=>", getattr(m, "supported_generation_methods", []))

# Filter to models that support text generation
gen_models = [m.name for m in genai.list_models()
              if "generateContent" in getattr(m, "supported_generation_methods", [])]
print("Text-capable models:", gen_models)


google-generativeai version: 0.8.5


E0000 00:00:1759651596.723686 14875709 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


models/embedding-gecko-001 => ['embedText', 'countTextTokens']
models/gemini-2.5-pro-preview-03-25 => ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.5-flash-preview-05-20 => ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.5-flash => ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.5-flash-lite-preview-06-17 => ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.5-pro-preview-05-06 => ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.5-pro-preview-06-05 => ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.5-pro => ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.0-flash-exp => ['generateContent', 'countTokens', 'bidiGenerateContent']
models/gemini-2.0-